<a href="https://colab.research.google.com/github/nmningmei/Deep_learning_fMRI_EEG/blob/master/7_2_simulation_of_statistical_learning_of_triplets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

# Let's generate some sequences of the triplets

In [0]:
import numpy as np
import pandas as pd
from collections import Counter

## The following generation of unique triplet items could be generalized to any number of unique items

In [0]:
# createing a list of random numbers from 1 to any
np.random.seed(12345) # set up a random seed
random12 = np.arange(1,28) # from 1 to 27
np.random.shuffle(random12)
random12 = random12.reshape(-1,3)
random12


array([[24,  9, 11],
       [19, 13, 22],
       [ 4, 14, 16],
       [20, 17, 21],
       [ 1,  7,  8],
       [12, 23, 15],
       [18, 25, 27],
       [26, 10,  5],
       [ 2,  6,  3]])

In [0]:
T_pos = {key:val for key,val in zip("ABC",random12)}
converting_map = {key:val for key,val in zip("ABC",np.arange(1,4))}

In [0]:
n_trials = 1500
n_repeats = 24
trials = np.zeros((n_trials,n_repeats*len(converting_map.keys()),3))
labels = np.zeros((n_trials,n_repeats*len(converting_map.keys())))

## let's make a loop to get one trial of sequence

In [0]:

try:
    trial = []
    counting = {key:n_repeats for key in T_pos.keys()}
    for ii in range(int(1e3)):
        
        if ii == 0:
            trial.append(np.random.choice(list(T_pos.keys())))
        
        elif ii > 0:
            last_pick = trial[ii - 1]
            pick_list = list(T_pos.keys())
            pick_list.remove(last_pick)
            pick_proba = np.array([counting[item] for item in pick_list])
            pick_proba = pick_proba / np.sum(pick_proba)
            trial.append(np.random.choice(pick_list,p = pick_proba))
        
        if len(trial) >= 4:
            focus = trial[-4:]
            group1 = focus[:2]
            group2 = focus[2:]
            if group1 == group2:
                trial[ii] = [item for item in list(T_pos.keys()) if (item != trial[ii - 1]) and (item != trial[ii - 2])][0]
            else:
                pass
        temp_counting = dict(Counter(trial))
        for key in temp_counting.keys():
            counting[key] = 24 - temp_counting[key]
        
except Exception as ex:
    print(ex)
    print(Counter(trial))

probabilities contain NaN
Counter({'A': 24, 'C': 24, 'B': 24})


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### As we rerun the cell above several times and then we would get something like `Counter({'B': 24, 'C': 24, 'A': 24})` and that is what we want. Since this works, let's make it a function

In [0]:
def generate_1_trial():
    try:
        trial = []
        counting = {key:n_repeats for key in T_pos.keys()}
        for ii in range(int(1e3)):

            if ii == 0:
                trial.append(np.random.choice(list(T_pos.keys())))

            elif ii > 0:
                last_pick = trial[ii - 1]
                pick_list = list(T_pos.keys())
                pick_list.remove(last_pick)
                pick_proba = np.array([counting[item] for item in pick_list])
                pick_proba = pick_proba / np.sum(pick_proba)
                trial.append(np.random.choice(pick_list,p = pick_proba))

            if len(trial) >= 4:
                focus = trial[-4:]
                group1 = focus[:2]
                group2 = focus[2:]
                if group1 == group2:
                    trial[ii] = [item for item in list(T_pos.keys()) if (item != trial[ii - 1]) and (item != trial[ii - 2])][0]
                else:
                    pass
            temp_counting = dict(Counter(trial))
            for key in temp_counting.keys():
                counting[key] = 24 - temp_counting[key]

    except Exception as ex:
        print(f"get {ex}, so repeat the process")
    return trial

### the reason I make this process a funtion is because I want to repeatedly calling this function until I get what I want, which is one trial of sequences

In [0]:
for _ in range(int(1e6)):
    trial = generate_1_trial()
    occurrence = dict(Counter(trial))
    occurrence = np.array([value for key,value in occurrence.items()])
    if all(occurrence == 24):
        print('done')
        break
print(Counter(trial),
     np.array(trial))
    

get probabilities are not non-negative, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities are not non-negative, so repeat the process
get probabilities contain NaN, so repeat the process
done
Counter({'B': 24, 'A': 24, 'C': 24}) ['B' 'A' 'C' 'A' 'B' 'A' 'C' 'B' 'A' 'B' 'C' 'B' 'A' 'B' 'C' 'A' 'C' 'B'
 'C' 'A' 'C' 'B' 'C' 'A' 'C' 'B' 'A' 'C' 'B' 'A' 'B' 'C' 'A' 'B' 'C' 'A'
 'C' 'B' 'A' 'C' 'A' 'B' 'A' 'C' 'B' 'C' 'A' 'B' 'C' 'B' 'A' 'C' 'B' 'C'
 'A' 'C' 'B' 'C' 'A' 'B' 'A' 'C' 'B' 'A' 'B' 'C' 'B' 'A' 'C' 'A' 'B' 'A']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### works well, let's start to generate many many trials

In [0]:
for_view = []
for n_trial_ in range(n_trials):
    for _ in range(int(1e6)):
        trial = generate_1_trial()
        occurrence = dict(Counter(trial))
        occurrence = np.array([value for key,value in occurrence.items()])
        if all(occurrence == 24):
            print('done')
            break
    trials[n_trial_,:,:] = [T_pos[item] for item in trial]
    labels[n_trial_,:] = [converting_map[item] for item in trial]
    for_view.append(trial)
for_view = np.array(for_view)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app


get probabilities contain NaN, so repeat the process
done
get probabilities contain NaN, so repeat the process
done
get probabilities contain NaN, so repeat the process
done
get probabilities contain NaN, so repeat the process
done
get probabilities contain NaN, so repeat the process
done
get probabilities contain NaN, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities contain NaN, so repeat the process
done
get probabilities contain NaN, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities are not non-negative, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities contain NaN, so repeat the process
get probabilities are not non-negative, so repeat the proces

### Let's randomly pick one of the trials and see if there are violations

In [0]:
idx_pick = np.random.choice(n_trials)
print(for_view[idx_pick])

['C' 'A' 'C' 'B' 'C' 'A' 'B' 'C' 'B' 'A' 'C' 'A' 'B' 'C' 'A' 'B' 'C' 'A'
 'C' 'B' 'C' 'A' 'B' 'A' 'C' 'B' 'C' 'A' 'B' 'C' 'B' 'A' 'B' 'C' 'B' 'A'
 'B' 'C' 'B' 'A' 'B' 'C' 'B' 'A' 'C' 'A' 'B' 'C' 'B' 'A' 'C' 'A' 'B' 'A'
 'C' 'B' 'A' 'C' 'A' 'B' 'C' 'A' 'B' 'C' 'A' 'C' 'B' 'A' 'C' 'A' 'B' 'A']


# the recurrent modeling part 1: supervised machine learning

In [0]:
import tensorflow as tf
from tensorflow.keras import (models,
                              layers,
                              optimizers,
                              regularizers,
                              losses,
                              metrics)
from sklearn.utils import shuffle

## First, we split the data into train (N = 1000), validation (N = 250), and test (N = 250)

In [0]:
sequences = trials.copy()
labels = labels.copy()

sequences, labels = shuffle(sequences,labels)

In [0]:
sequences_train,labels_train = sequences[:1000],labels[:1000]
sequences_valid,labels_valid = sequences[1000:1250],labels[1000:1250]
sequences_test,labels_test = sequences[1250:],labels[1250:]
print(sequences_train.shape,sequences_valid.shape,sequences_test.shape)
sequences_dict = {"train":sequences_train,
                  "valid":sequences_valid,
                  "test":sequences_test}
labels_dict = {"train":labels_train,
                  "valid":labels_valid,
                  "test":labels_test}

(1000, 72, 3) (250, 72, 3) (250, 72, 3)


## Second, we need to check the uniqueness of the sequential orders that do not overlap among train,validation, and test

In [0]:
converter = {key:val for key,val in zip(np.arange(1,4),"ABC")}

In [0]:
label_tran = {"train":set(),"valid":set(),"test":set()}
for key in label_tran.keys():
    for row in labels_dict[key]:
        temp = []
        for item in row:
            temp.append(converter[item])
        special = "".join(temp)
        label_tran[key].add(special)

In [0]:
for key,value in label_tran.items():
    print(f"{key} has {len(value)}")

train has 1000
valid has 250
test has 250


there is no repeated sequence order between train and validation

In [0]:
label_tran['train'].intersection(label_tran['valid'])

set()

there is no repeated sequence order between train and test

In [0]:
label_tran['train'].intersection(label_tran['test'])

set()

there is no repeated sequence order between test and validation

In [0]:
label_tran['test'].intersection(label_tran['valid'])

set()

## now, we define the RNN model

In [0]:
def build_RNN_model(batch_size = 10,drop = True):
    inputs = layers.Input(shape = sequences_train.shape[1:],
                          batch_size = batch_size,
                         name = 'inputs')
    RNN,state,seq = layers.LSTM(units = sequences_train.shape[1],
                                return_state = True,
                                return_sequences = True,
                                name = "RNN1")(inputs)
    RNN_2 = layers.LSTM(units = sequences_train.shape[1],
                        return_state = False,
                        return_sequences = True,
                        name = "RNN2")([RNN,state,seq])
    if drop:
        RNN_2 = layers.Dropout(0.5,name = "dropout")(RNN_2)
    outputs = layers.TimeDistributed(layers.Dense(3,name = 'outputs'))(RNN_2)
    outputs = layers.Activation('softmax',name ='softmax')(outputs)
    RNN_model = models.Model(inputs,outputs,name = "RNN")
    
    print(RNN_model.summary())
    return RNN_model

In [0]:
RNN_model = build_RNN_model()

W0626 19:09:37.683360 140328304838528 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "RNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(10, 72, 3)]        0                                            
__________________________________________________________________________________________________
RNN1 (LSTM)                     [(10, 72, 72), (10,  21888       inputs[0][0]                     
__________________________________________________________________________________________________
RNN2 (LSTM)                     (10, 72, 72)         41760       RNN1[0][0]                       
                                                                 RNN1[0][1]                       
                                                                 RNN1[0][2]                       
________________________________________________________________________________________________

## process the labels: one-hot encoding

In [0]:
labels_one_hot = {}
for key,values in labels_dict.items():
    temp = np.zeros(sequences_dict[key].shape)
    for ii,row in enumerate(values):
        for jj,item in enumerate(row):
            k = np.zeros(3)
            k[int(item - 1)] = 1
            temp[ii,jj,:] = k
    labels_one_hot[key] = np.array(temp)

In [0]:
RNN_model = build_RNN_model()
RNN_model.compile(optimizer = optimizers.Adam(lr = 1e-4),
                  loss = losses.MSE,
                  metrics = [metrics.kld])
n_epochs = 10
RNN_model.fit(sequences_dict['train'],
              labels_one_hot['train'],
              epochs = n_epochs,
              batch_size = batch_size,
              validation_data = (sequences_dict['valid'],
                                 labels_one_hot['valid']),
              )

Model: "RNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(10, 72, 3)]        0                                            
__________________________________________________________________________________________________
RNN1 (LSTM)                     [(10, 72, 72), (10,  21888       inputs[0][0]                     
__________________________________________________________________________________________________
RNN2 (LSTM)                     (10, 72, 72)         41760       RNN1[0][0]                       
                                                                 RNN1[0][1]                       
                                                                 RNN1[0][2]                       
________________________________________________________________________________________________

W0626 19:21:15.367600 140328304838528 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
1000/1000 [==============================] - 21s 21ms/sample - loss: 0.2127 - kullback_leibler_divergence: 1.0574 - val_loss: 0.1887 - val_kullback_leibler_divergence: 0.9549
Epoch 2/10
1000/1000 [==============================] - 19s 19ms/sample - loss: 0.1544 - kullback_leibler_divergence: 0.8071 - val_loss: 0.0914 - val_kullback_leibler_divergence: 0.5457
Epoch 3/10
1000/1000 [==============================] - 19s 19ms/sample - loss: 0.0527 - kullback_leibler_divergence: 0.3498 - val_loss: 0.0125 - val_kullback_leibler_divergence: 0.1505
Epoch 4/10
1000/1000 [==============================] - 20s 20ms/sample - loss: 0.0124 - kullback_leibler_divergence: 0.1285 - val_loss: 0.0041 - val_kullback_leibler_divergence: 0.0708
Epoch 5/10
1000/1000 [==============================] - 20s 20ms/sample - loss: 0.0060 - kullback_leibler_divergence: 0.0772 - val_loss: 0.0023 - val_kullback_leibler_divergence: 0.0469
Epoch 6/10
1000/1000 [==============================] - 19s 19ms/sampl

In [0]:
preds_valid = RNN_model.predict(sequences_dict['valid'],batch_size=batch_size)
preds_test = RNN_model.predict(sequences_dict['test'],batch_size=batch_size)


In [0]:
for a,b in zip(labels_one_hot['valid'].reshape(-1,3),preds_valid.reshape(-1,3)):
    print(f"one hot label = {a} probability = {b}")

one hot label = [0. 0. 1.] probability = [0.06769256 0.09172609 0.8405813 ]
one hot label = [0. 1. 0.] probability = [0.03025704 0.9352893  0.03445367]
one hot label = [0. 0. 1.] probability = [0.00493189 0.00974391 0.9853242 ]
one hot label = [1. 0. 0.] probability = [0.9773117  0.01870904 0.00397932]
one hot label = [0. 1. 0.] probability = [0.00759928 0.9817307  0.01067001]
one hot label = [0. 0. 1.] probability = [0.00244578 0.0054389  0.9921153 ]
one hot label = [1. 0. 0.] probability = [0.981659   0.01542628 0.00291468]
one hot label = [0. 0. 1.] probability = [0.00281951 0.01230434 0.98487616]
one hot label = [0. 1. 0.] probability = [0.00893051 0.9862153  0.00485413]
one hot label = [1. 0. 0.] probability = [0.9923832  0.00548743 0.00212947]
one hot label = [0. 0. 1.] probability = [0.00210935 0.01012908 0.9877616 ]
one hot label = [1. 0. 0.] probability = [0.979425   0.01707472 0.00350024]
one hot label = [0. 1. 0.] probability = [0.00470589 0.98624575 0.00904843]
one hot labe

In [0]:
for a,b in zip(labels_one_hot['test'].reshape(-1,3),preds_test.reshape(-1,3)):
    print(f"one hot label = {a} probability = {b}")

one hot label = [0. 1. 0.] probability = [0.070283   0.8602749  0.06944204]
one hot label = [0. 0. 1.] probability = [0.00711838 0.0180728  0.9748088 ]
one hot label = [0. 1. 0.] probability = [0.02217548 0.9666963  0.01112824]
one hot label = [1. 0. 0.] probability = [0.9924231  0.00564051 0.00193643]
one hot label = [0. 0. 1.] probability = [0.00356078 0.01288717 0.98355204]
one hot label = [0. 1. 0.] probability = [0.01224118 0.9825791  0.00517971]
one hot label = [1. 0. 0.] probability = [0.99294585 0.0051521  0.00190203]
one hot label = [0. 1. 0.] probability = [0.00441091 0.9869819  0.0086072 ]
one hot label = [0. 0. 1.] probability = [0.00221883 0.00418669 0.99359447]
one hot label = [1. 0. 0.] probability = [0.9865108  0.01115944 0.00232981]
one hot label = [0. 1. 0.] probability = [0.00467721 0.9857653  0.00955755]
one hot label = [0. 0. 1.] probability = [0.00212548 0.00412158 0.9937529 ]
one hot label = [1. 0. 0.] probability = [0.9867513  0.01081081 0.00243788]
one hot labe